In [1]:
# Импортирование библиотек для иницализации основных параметров системы
from fortran_calc_tools import Fazy, System, Garmoniki, Rachety, Looper, AttribHolder, PodStans, PhaseHolder

# Импортирование неодходимых библиотек
from os import path 
import time
import pandas as pd, numpy as np
from collections import namedtuple
from calc_tools import pairwise_1, pairwise_2
from scipy.linalg import logm, expm, sqrtm
from multiprocessing import Pool
from numpy.linalg import inv
from numpy import linalg as LA

# Импортирование библиотек для визуализации данных !!!!!
import matplotlib.pyplot as plot
import matplotlib as mpl
import seaborn as sns
import plotly as pt
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go

In [2]:
# Задание значении характеристик материала для фаз и троса
Faza_A = Fazy("Фаза А", 35.336, 1.0, 150.0, 0.0, 19.0)
Faza_B = Fazy("Фаза В", 35.336, 1.0, 150.0, 6.3, 19.0)
Faza_C = Fazy("Фаза С", 35.336, 1.0, 150.0, 4.2, 25.0)
Tross = Fazy("Тросс", 17.336, 4000.0, 50.0, 2.1, 28.0)

# В данном случае у нас будет 50 гармоник от [1 до 50]
garmoniki = Garmoniki(50)

# Создание объекта для проведения расчетов
rashet = Rachety()

# Инициализация объекта для подстанции "Зеленая" и чтение файла
basepath = path.abspath('')
filepath = path.abspath(path.join(basepath, "..", path.join("data", "Данные ПС Сарепта1.xls")))
PodS_zelenaya = PodStans(filepath, volt=110, harm_num=49)

dummy_str_input = "fuzzBuzz for testing"
dummy_int_input = 404

system = System(PodS_zelenaya, garmoniki, rashet, Fazy, dummy_str_input, dummy_str_input, dummy_int_input, dummy_int_input,
     dummy_int_input, dummy_int_input, dummy_int_input)

# Установка конфигурации для выведения данных
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [3]:
#                   !!! DEBUGGING PROCESS HERE !!!

In [4]:
UM, FUM, AIM, FIM, AU, PU, AI, PI, THD_U, THD_I, RMSU, RMSI = system.line_matrixWithPrisoed()

In [5]:
#                   !!! END OF DEBUGGING PROCESS !!!

In [ ]:
#                          !!! ВОТ И САМИ РАСЧЕТЫ !!!
class CalculateLosses():
   
    def __init__(self, podstan, prisoed=None, harm=None, t_izmer=None, distance=None, step=None):
        
        # Проверка на правильность введенных данных для расчета
        assert type(podstan) == PodStans, "Podstan must be PodStand object"
        assert type(prisoed) == int and prisoed != 0 and prisoed <= len(podstan.vse_prisoed), f"Prisoed number must be integer and not equal to zero. And within the range 1 and {len(podstan.vse_prisoed)}"
        if type(harm) == str:
            assert harm.upper() == "ALL", "Type 'ALL' to calculate all harmonics"
            self.all_harms = True
        else:
            assert type(harm) == int and harm < 50 and harm > 1, "Harmonic number must be integer and betweet values 2 and 49"
            self.all_harms = False
        assert type(distance) == int and distance != 0, "Distance number must be integer and not equal to zero"
        assert type(step) in [int, float] and step != 0, "Step size must be integer or float and not equal to zero"

        # Работа с определенным присоединением
        prisoed = getattr(podstan, "prisoed_"+str(prisoed))

        # Проверка налчичя данного измерения по времени в файле EXCEL
        self.num_record = len(prisoed.faza_B("U").index)
        assert t_izmer <= self.num_record and t_izmer >= 1, f"Time ticks must be within the range 1 and {self.num_record-1}"

        self.podstan = podstan
        self.prisoed = prisoed
        self.harm = harm
        self.t_izmer = t_izmer
        self.distance = distance
        self.step = step
        self.tros = 0
        
        self.result_dict = {
            "U_n": [],
            "U_k": [],
            "I_n": [],
            "I_k": [],
        }
        
        
    def prepare_columnsrows(self):
    # Составление векторов-столбцов тока и напряжения для начала линии второй гармоники 
        for i in range(self.t_izmer):
            if i == 0:
                volt_phaseA_har_n = self.prisoed.faza_A.get_harmonic(self.harm, 'U')[i]
                volt_phaseB_har_n = self.prisoed.faza_B.get_harmonic(self.harm, 'U')[i]
                volt_phaseC_har_n = self.prisoed.faza_C.get_harmonic(self.harm, 'U')[i]
                curr_phaseA_har_n = self.prisoed.faza_A.get_harmonic(self.harm, 'I')[i]
                curr_phaseB_har_n = self.prisoed.faza_B.get_harmonic(self.harm, 'I')[i]
                curr_phaseC_har_n = self.prisoed.faza_C.get_harmonic(self.harm, 'I')[i]

                U_n = np.array([[volt_phaseA_har_n], [volt_phaseB_har_n], [volt_phaseC_har_n], [0]])
                I_n = np.array([[curr_phaseA_har_n], [curr_phaseB_har_n], [curr_phaseC_har_n], [0]])
           

            volt_phaseA_har_k = self.prisoed.faza_A.get_harmonic(self.harm, 'U')[i]
            volt_phaseB_har_k = self.prisoed.faza_B.get_harmonic(self.harm, 'U')[i]
            volt_phaseC_har_k = self.prisoed.faza_C.get_harmonic(self.harm, 'U')[i]
            curr_phaseA_har_k = self.prisoed.faza_A.get_harmonic(self.harm, 'I')[i]
            curr_phaseB_har_k = self.prisoed.faza_B.get_harmonic(self.harm, 'I')[i]
            curr_phaseC_har_k = self.prisoed.faza_C.get_harmonic(self.harm, 'I')[i]

            U_k = np.array([[volt_phaseA_har_k], [volt_phaseB_har_k], [volt_phaseC_har_k], [0]])
            I_k = np.array([[curr_phaseA_har_k], [curr_phaseB_har_k], [curr_phaseC_har_k], [0]])

        
            # Нахождение векторов-столбцов постоянных интергирования
            E1 = np.identity(4)
            E2 = np.identity(4)
            E3 = np.identity(4)
            E4 = np.identity(4)

            pos_U_exponent, neg_U_exponent = rashet.calculate_exponentials(lambda_U, self.distance)
            pos_I_exponent, neg_I_exponent = rashet.calculate_exponentials(lambda_I, self.distance)

            GU = np.array([[[E1], [E2]],[[neg_U_exponent[self.harm-1]], [pos_U_exponent[self.harm-1]]]])
            GI = np.array([[[E3], [E4]],[[neg_I_exponent[self.harm-1]], [pos_I_exponent[self.harm-1]]]])

            GU_inv = inv(GU)
            GI_inv = inv(GI)

            self.Aw = np.dot(GU_inv[0][0][0], U_n).reshape(4,1) + np.dot(GU_inv[0][1][0], U_k).reshape(4,1)
            self.Bw = np.dot(GU_inv[1][0][0], U_n).reshape(4,1) + np.dot(GU_inv[1][1][0], U_k).reshape(4,1)
            self.Cw = np.dot(GI_inv[0][0][0], I_n).reshape(4,1) + np.dot(GI_inv[0][1][0], I_k).reshape(4,1)
            self.Dw = np.dot(GI_inv[0][0][0], I_n).reshape(4,1) + np.dot(GI_inv[0][1][0], I_k).reshape(4,1)
            
            
            Yw = inv(Y[self.harm-1])*-1
            Zw = inv(Z[self.harm-1])*-1

            lu = lambda_U[self.harm-1]
            li = lambda_I[self.harm-1]

            res_U_n = np.dot(Yw, (np.dot(-1*li, self.Cw) + np.dot(li, self.Dw)))
            res_I_n = np.dot(Zw, (np.dot(-1*lu, self.Aw) + np.dot(lu, self.Bw)))

            self.result_dict["U_n"].append(res_U_n)
            self.result_dict["I_n"].append(res_I_n)
            
            temp_I_holder = []
            temp_U_holder = []
            
            for step in range(1000, self.distance+1, 1000):
                pos_U_exponent, neg_U_exponent = rashet.calculate_exponentials(lambda_U, step)
                pos_I_exponent, neg_I_exponent = rashet.calculate_exponentials(lambda_I, step)
                
                res_U_k = np.dot(Yw, (np.dot(np.dot(-1*li, neg_I_exponent[self.harm-1]), self.Cw) + np.dot(np.dot(li, pos_I_exponent[self.harm-1]), self.Dw)))
                res_I_k = np.dot(Zw, (np.dot(np.dot(-1*lu, neg_U_exponent[self.harm-1]), self.Aw) + np.dot(np.dot(lu, pos_U_exponent[self.harm-1]), self.Bw)))
                
                temp_U_holder.append(res_U_k)
                temp_I_holder.append(res_I_k)
            
            self.result_dict["U_k"].append(temp_U_holder)
            self.result_dict["I_k"].append(temp_I_holder)               

        return self.result_dict    
            
    
    def calculate_losses(self, Is, phase):
        
        assert type(phase) == str, "Phase type must be defined as a string!"
        if phase.lower().strip() == "a":
            iphase = 0
        elif phase.lower().strip() == "b":
            iphase = 1
        elif phase.lower().strip() == "c":
            iphase = 2
        elif phase.lower().strip() == "t":
            iphase = 3
    
        phaseX = [(phases[iphase][0]) for phases in Is]; phase2sum = sum(list(map(lambda x:np.power(x,2), phaseX)))
        r = pogon_aktiv_soprotiv_faz[iphase][self.harm-1]
        overall_losses = r * phase2sum
        
        return overall_losses   


In [ ]:
if __name__ == "__main__":
#     start = time.time()
    calcs = CalculateLosses(PodS_zelenaya, prisoed=1, harm=2, t_izmer=560, distance=5000, step=1000)
    result_dict = calcs.prepare_columnsrows()
#     Losses = calcs.calculate_losses(result_dict["I_k"], "A")
#     end = time.time()
#     print("Execution time(sec): ", end - start)

In [ ]:
phase = 0 # Phase A
harm = 2 # Harmonic 2

all_vals = []
for time in range(560):
    temp_vals = []
    for step in range(5):
        temp_vals.append((result_dict["I_k"][time][step][phase].real**2) * pogon_aktiv_soprotiv_faz[phase][harm-1])
    
    all_vals.append(sum(temp_vals))
 

print(sum(all_vals))
for item in all_vals:
    print(item)

In [ ]:
for i, j in pairwise_1([1,2,3,4,5,6,7,8,9]):
    print(i,j)